In [26]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine, types
from sqlalchemy import text 

In [27]:
df_1 = pd.read_csv('../data/Cleaned/cleaned_world_marriage.csv')
df_2 = pd.read_csv('../data/Cleaned/cleaned_age_at_marriage_women.csv')
df_3 = pd.read_csv('../data/Cleaned/cleaned_marriage-rate-per-1000-inhabitants.csv')
df_4 = pd.read_csv('../data/Cleaned/cleaned_marriage-rates-in-1990-vs-2020.csv')
df_5 = pd.read_csv('../data/Cleaned/cleaned_share-of-births-outside-marriage.csv')
df_6 = pd.read_csv('../data/Cleaned/cleaned_share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv')
df_7 = pd.read_csv('../data/Cleaned/cleaned_share-of-single-parent-households.csv')
df_8 = pd.read_csv('../data/Cleaned/cleaned_share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv')


In [28]:
df_1.sample(5)

,country,age_group,sex,marital_status,data_process,data_collection_start_year,data_collection_end_year,data_source
243145,Thailand,[10-14],Woman,Married or in consensual union,Census,1970,1970,UNSD
235600,Sweden,[55-59],Man,Single,Estimate,1997,1997,UNSD
181322,Norway,[60-64],Woman,Divorced,Estimate,1979,1979,UNSD
178426,Niger,[30-34],Man,Married,Survey,2015,2015,National statistics
170297,Netherlands,[65-69],Woman,Married or married but separated,Estimate,1981,1981,UNSD


In [29]:
print(df_1.dtypes)

country                       object
age_group                     object
sex                           object
marital_status                object
data_process                  object
data_collection_start_year     int64
data_collection_end_year       int64
data_source                   object
dtype: object


In [30]:
df_2.sample(5)

,country,code,year,mean_age_of_women_at_first_marriage
532,Latvia,LVA,2009,26.90
497,Japan,JPN,1993,26.10
595,Luxembourg,LUX,2011,30.30
852,South Korea,KOR,1991,24.84
660,New Zealand,NZL,1997,26.50


In [31]:
df_2.dtypes

country                                 object
code                                    object
year                                     int64
mean_age_of_women_at_first_marriage    float64
dtype: object

In [32]:
df_3.sample(5)

,country,code,year,crude_marriage_rate_marriages_per_1000_people
1377,Luxembourg,LUX,1992,6.4
2184,Switzerland,CHE,1980,5.7
769,Germany,DEU,1967,7.8
710,Finland,FIN,1994,4.9
1345,Luxembourg,LUX,1960,7.1


In [33]:
df_4.sample(6)

,country,code,year,crude_marriage_rate,crude_marriage_rate_people1,year_1
1223,Lithuania,LTU,1966,9.2,9.8,1990
1548,New Zealand,NZL,2013,4.4,6.9,1990
814,Greece,GRC,2018,4.4,5.8,1990
1511,New Zealand,NZL,1976,7.7,6.9,1990
1687,Portugal,PRT,1969,9.0,7.2,1990
602,Estonia,EST,1988,8.3,7.5,1990


In [34]:
df_4.sample(6)

,country,code,year,crude_marriage_rate,crude_marriage_rate_people1,year_1
634,Estonia,EST,2020,4.6,7.5,1990
1885,Slovenia,SVN,1984,5.9,4.3,1990
721,Germany,DEU,1986,6.6,6.5,1990
1195,Latvia,LVA,1999,3.9,8.9,1990
2165,United Kingdom,GBR,1971,8.2,6.6,1990
495,Czechia,CZE,2003,4.8,8.8,1990


In [35]:
df_5.sample(5)

,country,code,year,share_of_births_outside_of_marriage
1729,Slovenia,SVN,1968,8.8
1758,Slovenia,SVN,1997,32.7
1835,Spain,ESP,1972,1.4
1037,Italy,ITA,1998,9.3
698,Germany,DEU,1977,9.0


In [36]:
df_6.sample(10)

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
59,Women,42,83.3,90.9,95.2,85.9,73.2
2,Men,19,0.8,0.6,2.0,2.5,0.7
47,Women,30,71.6,82.7,90.9,76.7,55.8
29,Men,46,92.1,91.4,92.1,80.8,68.8
17,Men,34,83.2,85.5,88.5,72.4,53.7
6,Men,23,21.4,26.8,38.1,26.2,10.5
12,Men,29,68.3,72.0,81.0,60.6,37.4
55,Women,38,81.2,89.8,94.6,84.5,70.5
25,Men,42,90.7,90.6,91.6,79.1,65.8
13,Men,30,72.9,76.4,83.3,63.9,41.4


In [37]:
df_7.sample(10)

,country,code,year,share_of_single_parent_households
180,Dominican Republic,DOM,2007,13.15
528,Palestine,PSE,2010,5.28
30,Azerbaijan,AZE,2009,8.40
207,Ethiopia,ETH,1994,13.03
120,Central African Republic,CAF,1994,6.27
136,Colombia,COL,1985,8.26
695,Tunisia,TUN,2011,8.75
221,French Polynesia,PYF,2007,7.45
373,Kosovo,OWID_KOS,2013,4.68
337,Italy,ITA,2011,9.91


In [38]:
df_8.sample(10)

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
67,Women,50,85.4,92.0,95.7,87.5,76.3
30,Men,47,92.3,91.6,92.2,81.0,69.4
57,Women,40,82.4,90.5,95.0,85.3,72.0
19,Men,36,86.1,87.6,89.7,74.8,58.3
47,Women,30,71.6,82.7,90.9,76.7,55.8
16,Men,33,81.5,84.1,87.6,70.8,51.1
64,Women,47,84.8,91.7,95.6,87.0,75.4
58,Women,41,82.9,90.7,95.1,85.7,72.7
9,Men,26,48.3,51.0,67.7,46.8,24.1
31,Men,48,92.5,91.7,92.3,81.3,69.9
